In [ ]:
from imports_scgen_local import * #imports_scgen_local have to be in same folder

In [ ]:
pre = 'MH68'
drive = 'F'
base_model_path,base_table_path,base_plots_path,base_anndata_objects = h.return_local_paths(drive = drive,
                                                                              pre = pre,
                                                                              add_path = True)
_,base_table_path_MH66_c,_,_ = h.return_local_paths(drive = drive,
                                                       pre = 'MH66_c',
                                                       add_path = False)

In [ ]:
base_table_path_MH66_c

In [ ]:
base_table_path_MH66_c = 'F:\\monkey_IZI\\analysis\\cluster_runs\\workspace-runVAE\\output\\MH66_c\\tables'
celltypes = ['B',
 'CD14 Mono',
 'CD16 Mono',
 'CD4 T',
 'CD8 T',
 'MAIT',
 'NK+Proliferating']
columns_lat_dim = [f'latent_dim_{i+1}' for i in range(10)]

In [ ]:
df_list = []
for celltype in celltypes:
    df_pert = pd.read_csv(os.path.join(base_table_path_MH66_c,'MH66_c_' + str(celltype) +'_pert.csv'),index_col=0)
    df_pert = df_pert.set_index('gene',drop=False)
    df_list.append(df_pert) 
df_pert_all = pd.concat(df_list)
df_pert_all['mean_perturbation_effect'] = df_pert_all[columns_lat_dim].mean(axis=1)
df_pert_all['median_perturbation_effect'] = df_pert_all[columns_lat_dim].median(axis=1)
#Save
df_pert_all.to_csv(os.path.join(base_table_path,pre + 'df_pert_all.csv'))

### Visualize

In [ ]:
top_genes_list = []
for celltype in celltypes:
    df_pert_all_ct = df_pert_all[df_pert_all['celltype'] == celltype]
    top_genes_list = top_genes_list + list(df_pert_all_ct.sort_values(by='mean_perturbation_effect',ascending=False).head(10)['gene'])

In [ ]:
unique_top_genes = pd.unique( top_genes_list)
df_dict = {}
df_sum_pert_per_group = {}
for celltype in celltypes:
    df_pert_all_ct = df_pert_all[(df_pert_all['celltype'] == celltype)]
    available_genes = [gene for gene in unique_top_genes if gene in df_pert_all_ct['gene'].values]
    df_sum_pert_per_group[celltype] = np.sum(df_pert_all_ct['mean_perturbation_effect'])
    df_dict[celltype] = df_pert_all_ct.loc[available_genes][['celltype','mean_perturbation_effect']]

In [ ]:
merged_df = pd.concat([pd.DataFrame(df_dict[celltype]['mean_perturbation_effect']).rename_axis('genes') for celltype in celltypes], 
                      axis=1, 
                      keys=celltypes,
                      sort=False).reset_index()

In [ ]:
df= pd.concat([pd.DataFrame(df_dict[celltype]['mean_perturbation_effect']).rename_axis('genes') for celltype in celltypes],axis=1)
df = df.fillna(0)
df.columns = celltypes
for celltype in celltypes:
    df[celltype] = (df[celltype]/df_sum_pert_per_group[celltype])*100

df_long = df.reset_index().melt(id_vars='genes', var_name='celltype', value_name='value')

In [ ]:
df_pivot = df_long.pivot('genes', 'celltype', 'value')
df_pivot = df_pivot[celltypes]
df_pivot = df_pivot.reindex(unique_top_genes)
#df_pivot.to_csv(os.path.join(save_loc_tables_rev06_visualization,species +  '_df_pivot_heatmap.csv'))
df_pivot.to_csv(os.path.join(base_table_path,pre + '_df_pivot.csv'))
plt.figure(figsize=(6, 15))
#sns.scatterplot(data=df_long, x='condition', y='genes', size='value', hue='value', legend=True, sizes=(30, 200))
sns.heatmap(df_pivot, annot=False, cmap='PuBuGn',linecolor='grey',
            linewidths=0.01)
plt.title('Effect of perturbing gene')
plt.xticks(rotation=90)
plt.savefig(os.path.join(base_plots_path,pre +'_heatmap_top_genes_VAE.pdf'),bbox_inches='tight')
plt.show()

In [ ]:
base_package_version_path = h.return_package_version_local_path(drive=drive)
h.save_package_versions(base_package_version_path,pre,do_print = True)